In [1]:
import pandas as pd

import codecs

import folium
import folium.plugins as plugins

from math import log

import warnings
warnings.filterwarnings("ignore")

In [2]:
def popup_html(data):
    html = '<font face="Helvetica" weight=400><h3>{}</h3>'.format(data.district)
    html += '<b>{}:</b> {} чел</p>'.format(name, data.value)
    return html

In [3]:
filename = 'data_district.xlsx'
df = pd.read_excel(filename)
df

,district,value
0,Воловский район,12161
1,Добровский район,23623
2,Грязинский район,79548
3,Данковский район,30466
4,Елецкий район,28415
5,Измалковский район,15250
6,Добринский район,32906
7,Долгоруковский район,16498
8,Задонский район,33872
9,Краснинский район,12099


In [4]:
lg = []
max_log = log(df.value.max())

for index in df.index:
    lg.append(max_log - log(df.loc[index].value))
    
df['log'] = lg

In [5]:
m = folium.Map([52.7, 39.1], zoom_start=8, tiles='cartodbpositron', control_scale=True)

filename = r'J:\~ 09_Машинное обучение_Прогноз показателей СЭР\Геограницы\Обновленные\Районы ЛО.geojson'
name = "Численность населения"

folium.Choropleth(
    geo_data=open(filename, encoding='utf-8').read(),
    name=name,
    data=df,
    key_on='feature.properties.name',
    columns=['district', 'log'],
    fill_color='RdYlGn',
    line_weight=1,
    fill_opacity=0.5,
    line_opacity=0.2,
).add_to(m)

for index in df.index:
    filename = r'J:\~ 09_Машинное обучение_Прогноз показателей СЭР\Геограницы\Обновленные\Районы ЛО\{}.geojson'.format(df.loc[index].district)
    geoj = folium.GeoJson(data=open(filename, encoding='utf-8').read(), control=False,
                          style_function = lambda x: {'color': 'black', 'fillOpacity' : 0, 'opacity' : 0.1, 'weight': 1},
                          highlight_function=lambda x: {'color': 'black', 'fillOpacity' : 0.25, 'opacity' : 0.1, 'weight': 1},
                          smooth_factor=0)
    iframe = folium.IFrame(popup_html(df.loc[index]), width=250, height=100)
    popup = folium.Popup(iframe)
    popup.add_to(geoj)
    geoj.add_to(m)

plugins.Fullscreen(position='topleft', title='На весь экран', title_cancel='Уменьшить', force_separate_button=True).add_to(m)

m.save('{} карта по районам ЛО.html'.format(name))